In [1]:
import tensorflow as tf
import os
from keras.layers import (Input, Dense, Flatten, Conv2D, MaxPooling2D, Dropout)
from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
import time

In [8]:
datapath = '../spectrograms3sec'
genres = list(os.listdir(f'{datapath}/train/'))
genres.remove('.DS_Store')
print(genres)

['pop', 'metal', 'disco', 'blues', 'reggae', 'classical', 'rock', 'hiphop', 'country', 'jazz']


In [9]:
#Data has been pre split into test and training folders

data_gen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)
path=f'{datapath}/train' 
image_datagentrain=data_gen.flow_from_directory(path,target_size=(300,300),class_mode='categorical')

Found 8990 images belonging to 10 classes.


In [10]:
path=f'{datapath}/test'
image_datagentest=data_gen.flow_from_directory(path,target_size=(300,300),class_mode='categorical')

Found 1000 images belonging to 10 classes.


In [11]:
def GenreModel(input_shape=(300,300,4), classes=10, model_name='GenreModel_'):
    
    model_name=model_name + str(int(time.time()))[4:]
    tensorboard = TensorBoard(log_dir=f'logs/{model_name}')

    X_input = Input(input_shape)
    
    X = (Conv2D(filters=8, kernel_size=(3,3), padding='same', activation='relu', input_shape=(300, 300, 3)))(X_input)
    X = (MaxPooling2D(pool_size=(2,2), strides=(2,2)))(X)

    X = (Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu',))(X)
    X = (MaxPooling2D(pool_size=(2,2), strides=(2,2)))(X)

    X = (Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu',))(X)
    X = (MaxPooling2D(pool_size=(2,2), strides=(2,2)))(X)

    X = (Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu',))(X)
    X = (MaxPooling2D(pool_size=(2,2), strides=(2,2)))(X)

    X = Flatten()(X)

    X = Dropout(rate=0.3)(X)

    X = (Dense(128, activation='relu'))(X)
    X = (Dense(64, activation='relu'))(X)
    X = (Dense(10, activation='softmax'))(X)

    model = Model(X_input,X)
    
    return model, tensorboard

model, tensorboard = GenreModel(input_shape=(300,300,3),classes=10)

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0005), metrics=['accuracy'])

history = model.fit(image_datagentrain,
                    epochs=25,
                    callbacks = [tensorboard],
                    batch_size=32,
                    validation_data=image_datagentest).history

Epoch 1/25
  1/281 [..............................] - ETA: 0s - loss: 2.3031 - accuracy: 0.0938WARNING:tensorflow:From /Users/jayvier/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
281/281 [==============================] - 149s 531ms/step - loss: 1.8587 - accuracy: 0.3158 - val_loss: 1.5462 - val_accuracy: 0.4400
Epoch 2/25
281/281 [==============================] - 158s 563ms/step - loss: 1.3376 - accuracy: 0.5196 - val_loss: 1.2651 - val_accuracy: 0.5560
Epoch 3/25
281/281 [==============================] - 156s 554ms/step - loss: 1.0620 - accuracy: 0.6297 - val_loss: 1.0727 - val_accuracy: 0.6210
Epoch 4/25
281/281 [==============================] - 163s 579ms/step - loss: 0.9272 - accuracy: 0.6825 - val_loss: 0.9038 - val_accuracy: 0.7040
Epoch 5/25
281/281 [================

In [12]:
loss, accuracy = model.evaluate(image_datagentest)
print('loss: ', loss, 'accuracy: ', accuracy)

32/32 [==============================] - 5s 158ms/step - loss: 0.8352 - accuracy: 0.7960
loss:  0.8352094888687134 accuracy:  0.7960000038146973


### TensorBoard
We can view an overview of our results in tensorboard.

In [13]:
from tensorboard import notebook

%reload_ext tensorboard
# Command line in current working dir "tensorboard --logdir='logs/'" to start tensorboard
%tensorboard --logdir {'logs/'}

Reusing TensorBoard on port 6008 (pid 607), started 1 day, 21:54:26 ago. (Use '!kill 607' to kill it.)

## Optimizing Model structure
Now that we have a basic model that works using hyperas we will find the optimal number of layers and layer sizes for the model, using tensorboard to visualise the results. It is possible that the current model is already optimal. To ealuate the models performance we will use the validation loss parameter. The hyperas library uses hyperopt, and allows us to feed the model a range fo possible parameters such as number of layers and number of nodes per layer.

In [14]:
import hyperas
from hyperas import optim
from hyperas.distributions import choice
from hyperopt import tpe, STATUS_OK, Trials
from keras.utils import np_utils

In [18]:
def GenreModel_optimize(X_train, Y_train, X_test, Y_test):

    X_input = Input((300,300,3))
    
    X = (Conv2D({{choice([8,16])}}, kernel_size=(3,3), padding='same', activation='relu', input_shape=(300, 300, 3)))(X_input)
    X = (MaxPooling2D(pool_size=(2,2), strides=(2,2)))(X)

    X = (Conv2D({{choice([8,16,32])}}, kernel_size=(3,3), padding='same', activation='relu',))(X)
    X = (MaxPooling2D(pool_size=(2,2), strides=(2,2)))(X)

    X = (Conv2D({{choice([16,32,64])}}, kernel_size=(3,3), padding='same', activation='relu',))(X)
    X = (MaxPooling2D(pool_size=(2,2), strides=(2,2)))(X)
    
    X = (Conv2D({{choice([32,64,128])}}, kernel_size=(3,3), padding='same', activation='relu',))(X)
    X = (MaxPooling2D(pool_size=(2,2), strides=(2,2)))(X)
    
    nb_conv = {{choice(['four','five'])}}
    if nb_conv == 'five':
        X = (Conv2D({{choice([64,128,256])}}, kernel_size=(3,3), padding='same', activation='relu',))(X)
        X = (MaxPooling2D(pool_size=(2,2), strides=(2,2)))(X)

    X = Flatten()(X)

    X = Dropout(rate=0.3)(X)
    
    nb_dense = {{choice(['one','two','three'])}}
    X = (Dense({{choice([64,128,256,512])}}, activation='relu'))(X)
    if nb_dense == 'two' or 'three':
        X = (Dense({{choice([32,64,128,256])}}, activation='relu'))(X)
        if nb_dense == 'three':
            X = (Dense({{choice([32,64,128,256])}}, activation='relu'))(X)

    
    X = (Dense(10, activation='softmax'))(X)

    model = Model(X_input,X)
    
    model.compile(loss='categorical_crossentropy',
                 optimizer=Adam(learning_rate=0.0005),
                 metrics=['accuracy'])
    
    batch_size = {{choice([32,64,128])}}
    history = model.fit(X_train,
                        epochs=20,
                        batch_size=batch_size,
                        validation_data=X_test).history
    
    
    
    return {'loss' : history['val_loss'][-1], 'status' : STATUS_OK, 'model' : model}

Hyperas requires a data function to feed to the `optim.minimize` funciton.

In [19]:
def data():
    nb_classes = 10
    
    data_gen=ImageDataGenerator(rescale=1./255.)

    X_train = data_gen.flow_from_directory('../spectrograms3sec/train',target_size=(300,300),class_mode='categorical')
    X_test = data_gen.flow_from_directory('../spectrograms3sec/test',target_size=(300,300),class_mode='categorical')
    
    y_train = X_train.classes
    y_test = X_test.classes
    
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)
    return X_train, Y_train, X_test, Y_test


We optimize the model using the optim function from hyperas.

In [20]:
X_train, Y_train, X_test, Y_test = data()

trials=Trials()

best_run, best_model = optim.minimize(model=GenreModel_optimize,
                                          data=data,
                                          max_evals=1,
                                          algo=tpe.suggest,
                                          notebook_name='CNNFinal',
                                          trials=trials)

print(best_model)

Found 8990 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
>>> Imports:
#coding=utf-8

try:
    import tensorflow as tf
except:
    pass

try:
    import os
except:
    pass

try:
    from keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
except:
    pass

try:
    from keras.models import Model, load_model
except:
    pass

try:
    from keras.optimizers import Adam
except:
    pass

try:
    from keras.preprocessing.image import ImageDataGenerator
except:
    pass

try:
    from keras.callbacks import TensorBoard
except:
    pass

try:
    import time
except:
    pass

try:
    from tensorboard import notebook
except:
    pass

try:
    import hyperas
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice
except:
    pass

try:
    from hyperopt import tpe, STATUS_OK, Trials
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

>>> Hyperas search

                                                    
 24/281 [=>............................]             
 - ETA: 2:46 - loss: 2.3072 - accuracy: 0.1172       
                                                    
 25/281 [=>............................]             
 - ETA: 2:46 - loss: 2.3070 - accuracy: 0.1163       
                                                    
 26/281 [=>............................]             
 - ETA: 2:46 - loss: 2.3066 - accuracy: 0.1166       
                                                    
 27/281 [=>............................]             
 - ETA: 2:45 - loss: 2.3070 - accuracy: 0.1134       
                                                    
 28/281 [=>............................]             
 - ETA: 2:45 - loss: 2.3071 - accuracy: 0.1127       
                                                    
 29/281 [==>...........................]             
 - ETA: 2:45 - loss: 2.3072 - accuracy: 0.1110       
                                  

 - ETA: 2:21 - loss: 2.3015 - accuracy: 0.1280       
                                                    
 75/281 [=======>......................]             
 - ETA: 2:20 - loss: 2.3007 - accuracy: 0.1296       
                                                    
 76/281 [=======>......................]             
 - ETA: 2:20 - loss: 2.3000 - accuracy: 0.1299       
                                                    
 77/281 [=======>......................]             
 - ETA: 2:19 - loss: 2.3005 - accuracy: 0.1299       
                                                    
 78/281 [=======>......................]             
 - ETA: 2:19 - loss: 2.2991 - accuracy: 0.1306       
                                                    
 79/281 [=======>......................]             
 - ETA: 2:18 - loss: 2.2985 - accuracy: 0.1313       
                                                    
 80/281 [=======>......................]             
 - ETA: 2:17 - loss: 2.2981 - accu

125/281 [============>.................]             
 - ETA: 1:48 - loss: 2.2332 - accuracy: 0.1595       
                                                    
126/281 [============>.................]             
 - ETA: 1:47 - loss: 2.2319 - accuracy: 0.1602       
                                                    
127/281 [============>.................]             
 - ETA: 1:47 - loss: 2.2284 - accuracy: 0.1622       
                                                    
128/281 [============>.................]             
 - ETA: 1:46 - loss: 2.2244 - accuracy: 0.1636       
                                                    
129/281 [============>.................]             
 - ETA: 1:45 - loss: 2.2207 - accuracy: 0.1664       
                                                    
130/281 [============>.................]             
 - ETA: 1:45 - loss: 2.2175 - accuracy: 0.1678       
                                                    
131/281 [============>............

                                                    
176/281 [=================>............]             
 - ETA: 1:13 - loss: 2.1351 - accuracy: 0.2088       
                                                    
177/281 [=================>............]             
 - ETA: 1:12 - loss: 2.1337 - accuracy: 0.2092       
                                                    
178/281 [==================>...........]             
 - ETA: 1:12 - loss: 2.1322 - accuracy: 0.2101       
                                                    
179/281 [==================>...........]             
 - ETA: 1:11 - loss: 2.1289 - accuracy: 0.2114       
                                                    
180/281 [==================>...........]             
 - ETA: 1:10 - loss: 2.1257 - accuracy: 0.2130       
                                                    
181/281 [==================>...........]             
 - ETA: 1:10 - loss: 2.1227 - accuracy: 0.2144       
                                  

...

 - ETA: 2:54 - loss: 0.0866 - accuracy: 0.9741         
                                                      
 54/281 [====>.........................]               
 - ETA: 2:53 - loss: 0.0869 - accuracy: 0.9734         
                                                      
 55/281 [====>.........................]               
 - ETA: 2:52 - loss: 0.0866 - accuracy: 0.9733         
                                                      
 56/281 [====>.........................]               
 - ETA: 2:52 - loss: 0.0864 - accuracy: 0.9732         
                                                      
 57/281 [=====>........................]               
 - ETA: 2:51 - loss: 0.0862 - accuracy: 0.9726         
                                                      
 58/281 [=====>........................]               
 - ETA: 2:50 - loss: 0.0867 - accuracy: 0.9720         
                                                      
 59/281 [=====>........................]              

 - ETA: 2:16 - loss: 0.0809 - accuracy: 0.9737         
                                                      
103/281 [=========>....................]               
 - ETA: 2:16 - loss: 0.0806 - accuracy: 0.9736         
                                                      
104/281 [==========>...................]               
 - ETA: 2:15 - loss: 0.0808 - accuracy: 0.9736         
                                                      
105/281 [==========>...................]               
 - ETA: 2:14 - loss: 0.0816 - accuracy: 0.9735         
                                                      
106/281 [==========>...................]               
 - ETA: 2:13 - loss: 0.0819 - accuracy: 0.9735         
                                                      
107/281 [==========>...................]               
 - ETA: 2:13 - loss: 0.0822 - accuracy: 0.9731         
                                                      
108/281 [==========>...................]              

 - ETA: 1:39 - loss: 0.0773 - accuracy: 0.9729         
                                                      
152/281 [===============>..............]               
 - ETA: 1:38 - loss: 0.0772 - accuracy: 0.9729         
                                                      
153/281 [===============>..............]               
 - ETA: 1:37 - loss: 0.0773 - accuracy: 0.9728         
                                                      
154/281 [===============>..............]               
 - ETA: 1:36 - loss: 0.0777 - accuracy: 0.9726         
                                                      
155/281 [===============>..............]               
 - ETA: 1:36 - loss: 0.0775 - accuracy: 0.9728         
                                                      
156/281 [===============>..............]               
 - ETA: 1:35 - loss: 0.0775 - accuracy: 0.9728         
                                                      
157/281 [===============>..............]              

 - ETA: 1:01 - loss: 0.0794 - accuracy: 0.9722         
                                                      
201/281 [====================>.........]               
 - ETA: 1:00 - loss: 0.0791 - accuracy: 0.9723         
                                                      
202/281 [====================>.........]               
 - ETA: 59s - loss: 0.0798 - accuracy: 0.9721          
                                                       
203/281 [====================>.........]               
 - ETA: 58s - loss: 0.0799 - accuracy: 0.9721          
                                                       
204/281 [====================>.........]               
 - ETA: 58s - loss: 0.0799 - accuracy: 0.9721          
                                                       
205/281 [====================>.........]               
 - ETA: 57s - loss: 0.0798 - accuracy: 0.9722          
                                                       
206/281 [====================>.........]          

249/281 [=========================>....]               
 - ETA: 23s - loss: 0.0761 - accuracy: 0.9741          
                                                       
250/281 [=========================>....]               
 - ETA: 23s - loss: 0.0761 - accuracy: 0.9741          
                                                       
251/281 [=========================>....]               
 - ETA: 22s - loss: 0.0758 - accuracy: 0.9742          
                                                       
252/281 [=========================>....]               
 - ETA: 21s - loss: 0.0757 - accuracy: 0.9742          
                                                       
253/281 [==========================>...]               
 - ETA: 20s - loss: 0.0756 - accuracy: 0.9742          
                                                       
254/281 [==========================>...]               
 - ETA: 20s - loss: 0.0754 - accuracy: 0.9743          
                                                

We see the results of the optimization by looking at the best run.

In [21]:
# 'Conv2D': hp.choice('Conv2D', [8,16]),
# 'Conv2D_1': hp.choice('Conv2D_1', [8,16,32]),
# 'Conv2D_2': hp.choice('Conv2D_2', [16,32,64]),
# 'Conv2D_3': hp.choice('Conv2D_3', [32,64,128]),
# 'nb_conv': hp.choice('nb_conv', ['four','five']),
# 'Conv2D_4': hp.choice('Conv2D_4', [64,128,256]),
# 'nb_dense': hp.choice('nb_dense', ['one','two','three']),
# 'Dense': hp.choice('Dense', [64,128,256,512]),
# 'Dense_1': hp.choice('Dense_1', [32,64,128,256]),
# 'Dense_2': hp.choice('Dense_2', [32,64,128,256]),
# 'batch_size': hp.choice('batch_size', [32,64,128]), (called Conv2D_5 in output)
best_run

{'Conv2D': 0,
 'Conv2D_1': 2,
 'Conv2D_2': 2,
 'Conv2D_3': 1,
 'Conv2D_4': 1,
 'Conv2D_5': 1,
 'Dense': 3,
 'Dense_1': 2,
 'Dense_2': 0,
 'nb_conv': 1,
 'nb_dense': 2}